In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Dense, Lambda,Embedding, Conv1D,Conv2D
from tensorflow.keras.layers import MaxPool1D,Dropout, Lambda, Concatenate, Flatten
from tensorflow.keras.layers import Multiply,Add
import matplotlib.pyplot as plt

In [ ]:
class dataprep:
  def __init__(self,num_targets, num_negs,seq_len):
    self.num_targets = num_targets
    self.num_negs = num_negs
    self.seq_len = seq_len

  def sequence_df(self):
    url = 'https://raw.githubusercontent.com/malinphy/datasets/main/ml100K/ratings.csv'
    df = pd.read_csv(url)
    df.rename(columns = {'userId':'user_id', 'movieId':'item_id'}, inplace = True)
  # df
    df['timestamp']= pd.to_datetime(df['timestamp'], unit = 's')
    df.head(10)

    def id_converter(df,x):
      unique_params = df[x].unique()
      num_unique = len(unique_params)
      param_2enc = { i:j for i,j in enumerate(unique_params)}
      enc_2param = { j:i for i,j in enumerate(unique_params)}


      return num_unique , param_2enc, enc_2param

    num_users, user_2enc ,enc_2user = id_converter(df,'user_id')
    num_items, item_2enc ,enc_2item = id_converter(df,'item_id')

    df['user_id'] = df['user_id'].map(enc_2user)
    df['item_id'] = df['item_id'].map(enc_2item)

    def time_sorter(df):
    ### users and releated items were sorted according to time as function of increasing time 
      return(df.set_index(['user_id','timestamp']).sort_index().reset_index())

    time_sorted_df = time_sorter(df)
    time_sorted_df = time_sorted_df.drop(columns= ['timestamp','rating'])   
    time_sorted_df.head(2)

    def users_grouped(df,x):
      return df.groupby([x]).aggregate(lambda tdf: tdf.unique().tolist()).reset_index()
    # df = df.reset_index()
    
    grouped_df = users_grouped(time_sorted_df,'user_id')
    grouped_df.head(3)

    # def sequence_creator(df, self.seq_len, self.num_targets):
    def sequence_creator(df):
      users = []
      seqs = []
      targets = []

      for i in range(len(grouped_df)):

        for j in range(0,len(grouped_df['item_id'][i])-self.seq_len,self.num_targets):
          users.append(i)
          x = grouped_df['item_id'][i][j:j+self.seq_len+self.num_targets]
          x1 = grouped_df['item_id'][i][j:j+self.seq_len]
          seqs.append(list(x[0:self.seq_len]))
          targets.append(np.array(x[-self.num_targets:]))

      return (pd.DataFrame({'users':users,'sequences':seqs, 'targets':targets}))

    L = 5
    d = 2
    sequenced_df = sequence_creator(grouped_df)
    sequenced_df.head(2)

    d = 2

    tot2 = []
    for j in range(self.num_negs):
      tot = []
      for i in range(len(sequenced_df)):
        sampled_neg = np.random.randint(num_items)
        x  = sequenced_df['users'][i]
        grouped_df['item_id'][x]
        while ((sampled_neg in sequenced_df['sequences'][i]) and  (sampled_neg in sequenced_df['sequences'][i] in grouped_df['item_id'][x] ) ):
          sampled_neg = np.random.randint(num_items)
        tot.append(sampled_neg)
  
      tot2.append(np.array(tot))

    negs = np.array(tot2).T

    sequenced_df['negs'] = list(negs)


    return sequenced_df, num_users, num_items

x = dataprep(1,1,5)
df,num_users, num_items = x.sequence_df()
df.head(1)

,users,sequences,targets,negs
0,0,"[16, 17, 19, 8, 9]",[10],[3605]


In [ ]:
test_locations = []
for i in df['users'].unique():
  # print(np.where(df['users'] == i)[0][-1])
  test_locations.append(np.where(df['users']==i)[0][-1])

print(len(df))

df_test = df.loc[test_locations,:]
df_train = df.drop(test_locations).reset_index()


96649


In [ ]:
def expander1(x):
    return (x[:,:,:,tf.newaxis])
    
def seqer2(x):
    return (tf.squeeze(x, axis=2))

def seqer1(x):
    return (tf.squeeze(x, axis=1))

TARGET_NUM = 1
batch_size = 4096
L=5
d=16
d_prime=4 
drop_ratio=0.05
num_factors= 10
dims = num_factors

In [ ]:
doublet = np.squeeze(np.dstack([df['targets'].astype('int'), df['negs'].astype('int')]))
k = []
sayac = 1
for i in df['sequences']:
  b =[]
  for j in i:
    b.append(int(j))
  k.append(b)

y_dummy = tf.ones( [len(df['users']),1])

In [ ]:
# user_input = tf.keras.Input(shape = (1,), name = 'user_input')  ### kullanildi
# seq_input = tf.keras.Input(shape = (L,), name = 'seq_input')  ### kullanildi
# doublet_input = tf.keras.Input(shape = (2,), name = 'doublet_input')

# seq_embedding = Embedding(num_items,num_factors, name = 'sequence_embedding')(seq_input)
# seq_embedding = tf.expand_dims(seq_embedding, 3)
# doublet_embedding = Embedding(num_items, num_factors*2,
#                               name = 'doublet_embedding')(doublet_input)## buyuk W
# doublet_embedding_b = Embedding(num_items, 1,
#                                name = 'doublet_embedding_1')(doublet_input)## kucuk b  
# user_embedding = Embedding(num_users, num_factors,
#                                name = 'user_embedding')(user_input)

# h = [i + 1 for i in range(L)]
# fc1_dim_v = d_prime * num_factors
# out_v = Conv2D(d_prime,(L,1), name = 'convo_v')(seq_embedding)
# out_v = Flatten(name = 'flatten_convo_v')(out_v)

# out_hs = []
# for i in (h):
  
#   seq_mod = tf.keras.Sequential([
#            Conv2D(d, (i, num_factors)),
#            Lambda(seqer2),
#            MaxPool1D(L - i + 1),
#            Lambda(seqer1)                      
#           ])
#   out_hs.append(seq_mod(seq_embedding))

# out_h = tf.concat(out_hs, axis=1) # galiba axis 2 olmali
# out = tf.concat([out_v, out_h], axis = 1)
# dropout_layer = Dropout(0.05, name = 'dropout_layer')(out)
# z = Dense(10, name = 'dense_layer')(dropout_layer)
# x = tf.concat([z, Flatten()(user_embedding)], axis=1)
# ####
# bir = tf.math.reduce_sum(x*tf.squeeze(doublet_embedding, axis=0, name='sqeu'),
#                          axis=1)# + b

# res2 = (tf.reshape(bir, [2,1])+ doublet_embedding_b)  
# res2 = tf.squeeze(res2,axis = [-1])                       
# # print('bir.shpae', bir.shape)
# # print(doublet_embedding.shape)
# # print('swq',tf.squeeze(doublet_embedding, axis=0, name='sqeu'
# # ).shape)
# # print(doublet_embedding_b.shape)
# # print('db',tf.squeeze(doublet_embedding_b, axis = 0, name='doublet_b'))
# # ####
# # print('x shape',x.shape)
# #### bu kisma kadar

# @tf.function
# def identity_loss(y_target, y_pred):
# # def identity_loss(y_target, y_pred):  
#   pos, neg = tf.split(y_pred,2,1)
#   # positive_loss = -1*tf.math.reduce_mean(tf.math.log(tf.sigmoid(pos)))
#   # negative_loss = -1*tf.math.reduce_mean(tf.math.log(1- tf.sigmoid(neg)))
#   # total_loss = tf.math.add(positive_loss, negative_loss)
#   return tf.math.reduce_mean(y_pred)
#   # return total_loss
# caser_model =tf.keras.Model( inputs= [
#                                       user_input,
#                                       seq_input,
#                                       doublet_input
#                                       ],
#                              outputs = res2 
#                             # outputs = x
#                             )
# caser_model.compile(
#     loss = identity_loss,
#     # loss = tf.keras.losses.CategoricalHinge(),

#     optimizer = 'Adam',
#     )




# print('x',x.shape)
# print('res2', res2.shape)
# caser_hist = caser_model.fit([tf.constant(df['users']),
#                             tf.constant(k),
#                 tf.constant(np.squeeze(np.dstack([df['targets'].astype('int'), df['negs'].astype('int')]))),
#                 # np.squeeze(np.dstack([df['targets'], df['negs']]))
#                 ],
#                 y_dummy
#                 , epochs = 1,
#                 batch_size = 20
#                 )

In [ ]:
user_input = tf.keras.Input(shape = (1,), name = 'user_input')  ### kullanildi
seq_input = tf.keras.Input(shape = (L,), name = 'seq_input')  ### kullanildi
doublet_input = tf.keras.Input(shape = (2,), name = 'doublet_input')

seq_embedding = Embedding(num_items,num_factors, name = 'sequence_embedding')(seq_input)
seq_embedding = tf.expand_dims(seq_embedding, 3, name = 'sequence_embedding_expander')
doublet_embedding = Embedding(num_items, num_factors*2,
                              name = 'doublet_embedding')(doublet_input)## buyuk W

doublet_embedding_b = Embedding(num_items, 1,
                               name = 'doublet_embedding_1')(doublet_input)## kucuk b  

user_embedding = Embedding(num_users, num_factors,
                               name = 'user_embedding')(user_input)

h = [i + 1 for i in range(L)]
fc1_dim_v = d_prime * num_factors
out_v = Conv2D(d_prime,(L,1), name = 'convo_v')(seq_embedding)
out_v = Flatten(name = 'flatten_convo_v')(out_v)

out_hs = []
for i in (h):
  seq_mod = tf.keras.Sequential([
           Conv2D(d, (i, num_factors)),
           Lambda(seqer2),
           MaxPool1D(L - i + 1),
           Lambda(seqer1)                      
          ])
  out_hs.append(seq_mod(seq_embedding))

out_h = tf.concat(out_hs, axis=1, name ='sequential_out') # galiba axis 2 olmali
out = tf.concat([out_v, out_h], axis = 1, name ='convo_concat')
dropout_layer = Dropout(0.05, name = 'dropout_layer')(out)
z = Dense(10, name = 'dense_layer')(dropout_layer)
x = tf.concat([z, Flatten()(user_embedding)], axis=1, name = 'x')
####
bir = tf.math.reduce_sum(x*tf.squeeze(doublet_embedding, axis=0, name='sqeu'),
                         axis=1)# + b

res2 = (tf.reshape(bir, [2,1])+ doublet_embedding_b)  
res2 = tf.squeeze(res2,axis = [-1])                       

@tf.function
def identity_loss(y_target, y_pred):
# def identity_loss(y_target, y_pred):  
  pos, neg = tf.split(y_pred,2,1)
  positive_loss = -1*tf.math.reduce_mean(tf.math.log(tf.sigmoid(pos)))
  negative_loss = -1*tf.math.reduce_mean(tf.math.log(1- tf.sigmoid(neg)))
  total_loss = tf.math.add(positive_loss, negative_loss)
  # return tf.math.reduce_mean(y_pred)
  return total_loss
caser_model =tf.keras.Model(inputs= [
                                      user_input,
                                      seq_input,
                                      doublet_input
                                      ],
                             outputs = res2)

caser_model.compile(loss = identity_loss,optimizer = 'Adam')

caser_hist = caser_model.fit(
                            [
                              tf.constant(df['users']),
                              tf.constant(k),
                              tf.constant( np.squeeze( np.dstack( [ df[ 'targets'].astype('int'), df['negs'].astype('int') ] ) ) ) ],
                y_dummy,
                epochs = 1,  
                batch_size = 1 ### BATCH SIZE DEGISINCE PROGRAM HATA VERIYOR, BUNU DUZELT
                )



96649/96649 [==============================] - 682s 7ms/step - loss: 1.0920


In [ ]:
print(out_h)

KerasTensor(type_spec=TensorSpec(shape=(None, 80), dtype=tf.float32, name=None), name='tf.concat_10/concat:0', description="created by layer 'tf.concat_10'")


In [ ]:
for layer in caser_model.layers:
    print(layer.name, layer)

seq_input <keras.engine.input_layer.InputLayer object at 0x7f183b333650>
sequence_embedding <keras.layers.embeddings.Embedding object at 0x7f183b333f90>
tf.expand_dims_4 <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7f183b341490>
convo_v <keras.layers.convolutional.Conv2D object at 0x7f183b338ed0>
sequential_20 <keras.engine.sequential.Sequential object at 0x7f183b35a250>
sequential_21 <keras.engine.sequential.Sequential object at 0x7f183b35f690>
sequential_22 <keras.engine.sequential.Sequential object at 0x7f183b2f4c50>
sequential_23 <keras.engine.sequential.Sequential object at 0x7f183b309450>
sequential_24 <keras.engine.sequential.Sequential object at 0x7f183b320850>
flatten_convo_v <keras.layers.core.flatten.Flatten object at 0x7f183b331390>
tf.concat_10 <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7f183b3254d0>
tf.concat_11 <keras.layers.core.tf_op_layer.TFOpLambda object at 0x7f183b236f90>
user_input <keras.engine.input_layer.InputLayer object at 0x7f183b3335d0>


In [ ]:
caser_model.get_layer('dense_layer').weights
# print((caser_model.get_layer('doublet_embedding').weights))

[<tf.Variable 'dense_layer/kernel:0' shape=(120, 10) dtype=float32, numpy=
 array([[-2.2053210e-01,  3.8467854e-02,  3.2171238e-02, ...,
          8.7354653e-02,  1.5342752e-03,  4.8971742e-02],
        [-2.1870284e-01,  3.0579757e-02,  2.6617449e-02, ...,
         -3.1368500e-01, -3.5402775e-02,  2.3689847e-02],
        [-4.1116127e-05,  6.5744512e-02, -1.5977874e-01, ...,
          4.1012902e-02, -2.4211276e-01, -1.2739694e-01],
        ...,
        [-3.0672597e-02, -9.6873432e-02, -6.6502705e-02, ...,
          1.5705855e-01,  4.9339559e-02, -1.1708725e-01],
        [ 1.8776618e-02, -5.9855349e-02, -1.0008163e-01, ...,
          7.8845941e-02, -4.2184908e-02, -6.2889278e-02],
        [-7.3913917e-02,  1.4212255e-01,  7.9315230e-02, ...,
          1.1877959e-01,  1.2160551e-02, -7.0107281e-02]], dtype=float32)>,
 <tf.Variable 'dense_layer/bias:0' shape=(10,) dtype=float32, numpy=
 array([-0.12880032,  0.00959405, -0.0236844 ,  0.01437702,  0.03926595,
         0.06203525, -0.10160863

In [ ]:
# def predict(model, user_id, sequence):
print((caser_model.get_layer('doublet_embedding').weights))

[<tf.Variable 'doublet_embedding/embeddings:0' shape=(9066, 20) dtype=float32, numpy=
array([[ 0.491121  ,  0.07382999,  0.16407499, ..., -0.01226526,
        -0.00761332,  0.05060099],
       [ 0.31008473,  0.68797106,  0.39555055, ..., -0.40286288,
        -0.31379974, -0.03876671],
       [ 0.09514315,  0.2158661 ,  0.22104913, ..., -0.10127037,
         0.31795242,  0.23064938],
       ...,
       [ 0.04864021, -0.15050004, -0.13373794, ...,  0.2117553 ,
         0.14202695, -0.04507238],
       [ 0.11061231, -0.18267606, -0.14490828, ..., -0.07370197,
         0.00160368,  0.01920417],
       [-0.06880045, -0.23289673, -0.41206938, ...,  0.23468111,
         0.01205959, -0.3699207 ]], dtype=float32)>]


In [ ]:
# w = w
tf.matmul(caser_model.get_layer('dense_layer').weights,
          tf.squeeze(tf.transpose(caser_model.get_layer('doublet_embedding').weights),axis=[-1]))

InvalidArgumentError: ignored

In [ ]:
tf.squeeze(tf.transpose(caser_model.get_layer('doublet_embedding').weights),axis=[-1])

<tf.Tensor: shape=(20, 9066), dtype=float32, numpy=
array([[ 0.491121  ,  0.31008473,  0.09514315, ...,  0.04864021,
         0.11061231, -0.06880045],
       [ 0.07382999,  0.68797106,  0.2158661 , ..., -0.15050004,
        -0.18267606, -0.23289673],
       [ 0.16407499,  0.39555055,  0.22104913, ..., -0.13373794,
        -0.14490828, -0.41206938],
       ...,
       [-0.01226526, -0.40286288, -0.10127037, ...,  0.2117553 ,
        -0.07370197,  0.23468111],
       [-0.00761332, -0.31379974,  0.31795242, ...,  0.14202695,
         0.00160368,  0.01205959],
       [ 0.05060099, -0.03876671,  0.23064938, ..., -0.04507238,
         0.01920417, -0.3699207 ]], dtype=float32)>